<a href="https://colab.research.google.com/github/positivejmk/2021-2-practice/blob/master/%EA%B8%88%2C%ED%99%98%EC%9C%A8%2C%EC%BD%94%EC%8A%A4%ED%94%BC%EC%A7%80%EC%88%98%2C%EC%BD%94%EC%8A%A4%EB%8B%A5%EC%A7%80%EC%88%98_%EB%A0%88%ED%94%84%ED%8A%B8%EC%A1%B0%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os
import matplotlib.pyplot as plt


from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2021년(제2회) NH투자증권 빅데이터 경진대회/nh prediction.csv")
gold_usd = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/2021년(제2회) NH투자증권 빅데이터 경진대회/gold_usd_20160104-20201229")
kospi_kosdaq = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/2021년(제2회) NH투자증권 빅데이터 경진대회/KOSPI_KOSDAQ_20160104-20201229.xlsx")

In [ ]:
df.head()

,Unnamed: 0,act_id,iem_cd,byn_dt,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,11,1,9,3,2,2,9,5,5,8,101,1,1,1,20180726,274.0,11782000.0,5000.0,43000.0
1,6,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,2018-01-31,80,1,9,3,2,2,9,5,5,8,1361,9,1,1,20180131,2.0,4990000.0,5000.0,2495000.0
2,15,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,2018-05-17,5,1,9,3,2,2,9,5,5,8,2530,12,2,99,20180517,786.0,14619600.0,1000.0,18600.0
3,18,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,2020-11-12,22,1,9,3,2,2,9,5,5,8,1969,8,2,1,20201112,60.0,462000.0,500.0,7700.0
4,20,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,2018-09-05,324,1,9,3,2,2,9,5,5,8,1696,10,3,99,20180905,183.0,8125200.0,5000.0,44400.0


In [ ]:
gold_usd.head()

,일자,금가격,원달러환율
0,NaT,(USD/트로이온즈),(원/USD)
1,2020-12-29,1879.14,1094.5
2,2020-12-28,1883.94,1096.5
3,2020-12-24,1878.14,1103
4,2020-12-23,1868.69,1107


In [ ]:
kospi_kosdaq.head()

,시점,KOSPI지수 (1980.01.04=100),"거래량(주식시장, 잠정치) (만주)","거래대금(주식시장 , 잠정치) (억원)","외국인 순매수(주식시장, 잠정치) (억원)","주식시장-거래량(만주, 시간외거래분 포함) (만주)","주식시장-거래대금(억원, 시간외거래분 포함) (억원)","시가총액(주식시장, 잠정치) (억원)",KOSDAQ지수 (1996.07.01=1000),"거래량(만주 : 코스닥시장, 잠정치) (만주)","거래대금(억원 : 코스닥시장, 잠정치) (억원)","외국인 순매수(코스닥시장, 잠정치) (억원)"
0,2016. 01. 04,1918.76,35500,39173,-1572,35902,39663,12153840,677.79,50271,31183,-228
1,2016. 01. 05,1930.53,44091,40980,-1897,44655,41976,12226220,684.07,74169,32932,64
2,2016. 01. 06,1925.43,58598,59180,1631,59461,62747,12192970,687.27,74761,41334,-46
3,2016. 01. 07,1904.33,38814,49072,-2693,39304,50030,12055460,679.66,69215,37943,-157
4,2016. 01. 08,1917.62,40056,49954,-1492,43020,52055,12137390,682.56,71075,37161,245


In [ ]:
gold_usd = gold_usd[1:]

In [ ]:
gold_usd = gold_usd.rename(columns={'일자':'byn_dt'})

In [ ]:
kospi_kosdaq = kospi_kosdaq.rename(columns={'시점':'byn_dt'})

In [ ]:
gold_usd.head()

,byn_dt,금가격,원달러환율
1,2020-12-29,1879.14,1094.5
2,2020-12-28,1883.94,1096.5
3,2020-12-24,1878.14,1103
4,2020-12-23,1868.69,1107
5,2020-12-22,1871.79,1107.5


In [ ]:
kospi_kosdaq.head()

,byn_dt,KOSPI지수 (1980.01.04=100),"거래량(주식시장, 잠정치) (만주)","거래대금(주식시장 , 잠정치) (억원)","외국인 순매수(주식시장, 잠정치) (억원)","주식시장-거래량(만주, 시간외거래분 포함) (만주)","주식시장-거래대금(억원, 시간외거래분 포함) (억원)","시가총액(주식시장, 잠정치) (억원)",KOSDAQ지수 (1996.07.01=1000),"거래량(만주 : 코스닥시장, 잠정치) (만주)","거래대금(억원 : 코스닥시장, 잠정치) (억원)","외국인 순매수(코스닥시장, 잠정치) (억원)"
0,2016. 01. 04,1918.76,35500,39173,-1572,35902,39663,12153840,677.79,50271,31183,-228
1,2016. 01. 05,1930.53,44091,40980,-1897,44655,41976,12226220,684.07,74169,32932,64
2,2016. 01. 06,1925.43,58598,59180,1631,59461,62747,12192970,687.27,74761,41334,-46
3,2016. 01. 07,1904.33,38814,49072,-2693,39304,50030,12055460,679.66,69215,37943,-157
4,2016. 01. 08,1917.62,40056,49954,-1492,43020,52055,12137390,682.56,71075,37161,245


In [ ]:
df['byn_dt'] = pd.to_datetime(df['byn_dt'])
gold_usd['byn_dt'] = pd.to_datetime(gold_usd['byn_dt'])
kospi_kosdaq['byn_dt'] = pd.to_datetime(kospi_kosdaq['byn_dt'])

In [ ]:
df_final = pd.merge(df, gold_usd, how = "left", on = ["byn_dt"])

In [ ]:
df_final = pd.merge(df_final, kospi_kosdaq, how = "left", on = ["byn_dt"])

In [ ]:
df_final.head()

,Unnamed: 0,act_id,iem_cd,byn_dt,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bse_dt,bnc_qty,tot_aet_amt,stk_par_pr,stk_p,금가격,원달러환율,KOSPI지수 (1980.01.04=100),"거래량(주식시장, 잠정치) (만주)","거래대금(주식시장 , 잠정치) (억원)","외국인 순매수(주식시장, 잠정치) (억원)","주식시장-거래량(만주, 시간외거래분 포함) (만주)","주식시장-거래대금(억원, 시간외거래분 포함) (억원)","시가총액(주식시장, 잠정치) (억원)",KOSDAQ지수 (1996.07.01=1000),"거래량(만주 : 코스닥시장, 잠정치) (만주)","거래대금(억원 : 코스닥시장, 잠정치) (억원)","외국인 순매수(코스닥시장, 잠정치) (억원)"
0,0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,11,1,9,3,2,2,9,5,5,8,101,1,1,1,20180726,274.0,11782000.0,5000.0,43000.0,1228.59,1120,2289.06,29532,61114,1285,30333,63571,15309140,765.15,53644,35128,591
1,6,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,2018-01-31,80,1,9,3,2,2,9,5,5,8,1361,9,1,1,20180131,2.0,4990000.0,5000.0,2495000.0,1342.1,1067,2566.46,39690,107348,-6630,40305,108426,16685790,913.57,119526,75432,-1782
2,15,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,2018-05-17,5,1,9,3,2,2,9,5,5,8,2530,12,2,99,20180517,786.0,14619600.0,1000.0,18600.0,1287.43,1081.5,2448.45,58898,80783,-2708,59677,81905,16345480,855.62,103744,54606,729
3,18,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,2020-11-12,22,1,9,3,2,2,9,5,5,8,1969,8,2,1,20201112,60.0,462000.0,500.0,7700.0,1867.5,1115.5,2475.62,125045,157200,4898,127869,164200,16983290,840.08,206889,110048,63
4,20,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,2018-09-05,324,1,9,3,2,2,9,5,5,8,1696,10,3,99,20180905,183.0,8125200.0,5000.0,44400.0,1195.25,1123.5,2291.77,35203,52202,-980,35819,53905,15333130,820.00,66369,43497,256
